In [ ]:
"""
Created on Thu Oct 25 09:54:12 2018

@author: LongJun
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import config as cfg
import os
import pascal_voc as pascl
import tensorflow.contrib.slim as slim
import anchor_generate
from tensorflow.python import pywrap_tensorflow
from anchor_label import anchor_labels_process, labels_generate, anchor_labels_process
import numpy as np
import tensorflow as tf
import network
import datetime
from losslayer import RPN_loss
from predict_loss import Predict_loss
# Solver Class, used for training
# net: the name of backbone net, only support VGG16, more backbone net will be supported in the feature
# data/val_data： trian_data/vla_data is a list that consist of dict of ground_truth label
# rpn_loss class : used for calculating rpn_loss
# predict_loss: used for calculating predict_loss
class Solver(object):   
    def __init__(self, net ,data, val_data, rpn_loss, predict_loss): 
        self.net = net
        self.data = data
        self.val_data = val_data
        self.max_iter = cfg.MAX_ITER
        self.lr = cfg.LEARNING_RATE
        self.rpn_loss = rpn_loss
        self.predict_loss = predict_loss
        self.lr_change_ITER = cfg.lr_change_ITER
        self.summary_iter = cfg.SUMMARY_ITER
        self.save_iter = cfg.SAVE_ITER
        self.overlaps_max = cfg.overlaps_max
        self.overlaps_min = cfg.overlaps_min
        self._variables_to_fix = {}
        self.Summary_output = os.path.join(cfg.Summary_output, datetime.datetime.now().strftime('%Y_%m_%d_%H_%M'))
        if not os._exists(self.Summary_output):
            os.mkdir(self.Summary_output)
        self.train_summary_dir = os.path.join(self.Summary_output, 'train')
        self.val_summary_dir = os.path.join(self.Summary_output, 'val')
        self.model_output_dir = os.path.join(cfg.OUTPUT_DIR) 
        if not os.path.exists(self.model_output_dir):
            os.mkdir(self.model_output_dir)
        if not os.path.exists(self.train_summary_dir):
            os.mkdir(self.train_summary_dir)
        if not os.path.exists(self.val_summary_dir):
            os.mkdir(self.val_summary_dir)
        self.ckpt_filename = os.path.join(self.model_output_dir, 'output.model')
        
 # training process       
    def train_model(self):
        lr = tf.Variable(self.lr[0],trainable=False)
        self.optimizer = tf.train.MomentumOptimizer(lr, cfg.momentum)
        #self.optimizer = tf.train.GradientDescentOptimizer(lr)
        self.loss = self.rpn_loss.add_loss() + self.predict_loss.add_loss()     
        train_op = self.optimizer.minimize(self.loss)
        variables = tf.global_variables()
        reader = pywrap_tensorflow.NewCheckpointReader(self.net.weight_file_path)
        var_to_shape_map = reader.get_variable_to_shape_map()
        variables_to_restore = self.get_var_list(variables, var_to_shape_map)
        init = tf.global_variables_initializer()
        saver = tf.train.Saver(var_list=variables_to_restore)
        merged = tf.summary.merge_all()
        with tf.Session() as sess:
            train_writer = tf.summary.FileWriter(self.train_summary_dir, sess.graph)
            val_writer = tf.summary.FileWriter(self.val_summary_dir)
            sess.run(init)
            saver.restore(sess, self.net.weight_file_path)
            self.fix_variables(sess, self.net.weight_file_path)
            saver = tf.train.Saver(variables,max_to_keep = 10)
            for step in range(self.max_iter+1):
                if step == self.lr_change_ITER:
                    lr = tf.assign(lr, self.lr[1])
                train_data = self.data.get()
                image_height = np.array(train_data['image'].shape[1])
                image_width = np.array(train_data['image'].shape[2])
                feed_dict = {self.net.image: train_data['image'], self.net.image_width: image_width,\
                             self.net.image_height: image_height, self.net.gt_boxes: train_data['box'],\
                             self.net.gt_cls: train_data['cls']}
                if step % self.summary_iter == 0:
                    total_loss, summary, learning_rate= sess.run([self.loss, merged, lr], feed_dict=feed_dict)
                    train_writer.add_summary(summary, step)
                    val_data = self.val_data.get()
                    val_image_height = np.array(val_data['image'].shape[1])
                    val_image_width = np.array(val_data['image'].shape[2])
                    val_feed_dict = {self.net.image: val_data['image'], self.net.image_width: val_image_width,\
                                     self.net.image_height: val_image_height, self.net.gt_boxes: val_data['box'],\
                                     self.net.gt_cls: val_data['cls']}
                    val_loss, val_summary = sess.run([self.loss, merged], feed_dict=val_feed_dict)
                    val_writer.add_summary(val_summary, step)
                    print ('The', step, 'step train_total_loss is', total_loss, 'val_total_loss is', val_loss)
                    print ('learning_rate is ', learning_rate)
                if step % self.save_iter == 0:
                    saver.save(sess, self.ckpt_filename, global_step = step)
                sess.run(train_op, feed_dict=feed_dict)
                    
               
                
                
#get the variables to restore               
    def get_var_list(self, global_variables, ckpt_variables):
        variables_to_restore = []
        for key in global_variables:
            print (key.name)
            if key.name == ('vgg_16/fc6/weights:0') or key.name == ('vgg_16/fc7/weights:0'):
                self._variables_to_fix[key.name] = key
                continue
            
            if key.name.split(':')[0] in ckpt_variables:
                variables_to_restore.append(key) 
        return variables_to_restore
    
#because fc6 and fc7 layers of pretrained vgg16 model is convolution format, so we need convert them to fully-connected layers
    def fix_variables(self, sess, pretrained_model):
        print('Fix VGG16 layers..')
        with tf.variable_scope('Fix_VGG16') as scope:
            with tf.device("/cpu:0"):
                fc6_conv = tf.get_variable("fc6_conv", [7, 7, 512, 4096], trainable=False)
                fc7_conv = tf.get_variable("fc7_conv", [1, 1, 4096, 4096], trainable=False)
                restorer_fc = tf.train.Saver({'vgg_16' + "/fc6/weights": fc6_conv, 
                                              'vgg_16' + "/fc7/weights": fc7_conv})
                restorer_fc.restore(sess, pretrained_model)
        
                sess.run(tf.assign(self._variables_to_fix['vgg_16' + '/fc6/weights:0'], tf.reshape(fc6_conv, 
                                    self._variables_to_fix['vgg_16' + '/fc6/weights:0'].get_shape())))
                sess.run(tf.assign(self._variables_to_fix['vgg_16' + '/fc7/weights:0'], tf.reshape(fc7_conv, 
                                    self._variables_to_fix['vgg_16' + '/fc7/weights:0'].get_shape())))
    

if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID
    net = network.Net()
    rpn_loss_obj = RPN_loss(net.rois_output['rois_bbx'], net.all_anchors, net.gt_boxes, \
                        net.rois_output['rois_cls'], net.labels, net.anchor_obj)
    predict_loss = Predict_loss(net._predictions["cls_score"], net._proposal_targets['labels'],\
                                net._predictions['bbox_pred'], net._proposal_targets['bbox_targets'],\
                                net._proposal_targets['bbox_inside_weights'], net._proposal_targets['bbox_outside_weights'])
    
    train_data = pascl.pascal_voc(cfg.train_imdb_name, 'train', fliped=True)
    val_data = pascl.pascal_voc(cfg.test_imdb_name, 'test', fliped=False)
    solver = Solver(net, train_data, val_data, rpn_loss_obj, predict_loss)
    print ('start training')
    start = datetime.datetime.now()
    solver.train_model()
    end = datetime.now()
    latency = (end - start).total_seconds()  
    print("The latency is   "+str(latency)+"秒")  
    
    
    
    
    
    
    
    
    

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differen

The 900 step train_total_loss is 0.7260959 val_total_loss is 0.5231265
learning_rate is  0.001
The 950 step train_total_loss is 0.17131145 val_total_loss is 0.52775455
learning_rate is  0.001
The 1000 step train_total_loss is 0.42995507 val_total_loss is 0.41908002
learning_rate is  0.001
The 1050 step train_total_loss is 0.58942294 val_total_loss is 0.3054273
learning_rate is  0.001
The 1100 step train_total_loss is 0.16183375 val_total_loss is 0.16329779
learning_rate is  0.001
The 1150 step train_total_loss is 0.04562131 val_total_loss is 0.5633826
learning_rate is  0.001
The 1200 step train_total_loss is 1.1555414 val_total_loss is 0.76915216
learning_rate is  0.001
The 1250 step train_total_loss is 0.30301613 val_total_loss is 0.5144563
learning_rate is  0.001
The 1300 step train_total_loss is 0.19840613 val_total_loss is 1.0491308
learning_rate is  0.001
The 1350 step train_total_loss is 0.76914036 val_total_loss is 0.6052792
learning_rate is  0.001
The 1400 step train_total_loss

The 5150 step train_total_loss is 0.47906002 val_total_loss is 0.7076856
learning_rate is  0.001
The 5200 step train_total_loss is 0.2790257 val_total_loss is 0.1587047
learning_rate is  0.001
The 5250 step train_total_loss is 0.18186711 val_total_loss is 1.1643443
learning_rate is  0.001
The 5300 step train_total_loss is 0.22621053 val_total_loss is 0.461559
learning_rate is  0.001
The 5350 step train_total_loss is 0.30772755 val_total_loss is 0.12262987
learning_rate is  0.001
The 5400 step train_total_loss is 1.0604599 val_total_loss is 1.0490677
learning_rate is  0.001
The 5450 step train_total_loss is 0.6234527 val_total_loss is 0.24255753
learning_rate is  0.001
The 5500 step train_total_loss is 0.492837 val_total_loss is 1.9687893
learning_rate is  0.001
The 5550 step train_total_loss is 0.7544813 val_total_loss is 0.40978765
learning_rate is  0.001
The 5600 step train_total_loss is 0.5450933 val_total_loss is 0.3365354
learning_rate is  0.001
The 5650 step train_total_loss is 0

The 9400 step train_total_loss is 0.5313782 val_total_loss is 0.042587824
learning_rate is  0.001
The 9450 step train_total_loss is 0.3839687 val_total_loss is 0.97541434
learning_rate is  0.001
The 9500 step train_total_loss is 0.9440693 val_total_loss is 0.3669292
learning_rate is  0.001
The 9550 step train_total_loss is 0.47466373 val_total_loss is 0.3298045
learning_rate is  0.001
The 9600 step train_total_loss is 0.14064941 val_total_loss is 0.26822844
learning_rate is  0.001
The 9650 step train_total_loss is 0.2650977 val_total_loss is 0.5355016
learning_rate is  0.001
The 9700 step train_total_loss is 0.24714887 val_total_loss is 0.60991895
learning_rate is  0.001
The 9750 step train_total_loss is 0.67007375 val_total_loss is 0.28382748
learning_rate is  0.001
The 9800 step train_total_loss is 0.45666745 val_total_loss is 0.52068746
learning_rate is  0.001
The 9850 step train_total_loss is 0.4681353 val_total_loss is 0.1474392
learning_rate is  0.001
The 9900 step train_total_lo

The 13600 step train_total_loss is 0.15934232 val_total_loss is 0.094983175
learning_rate is  0.001
The 13650 step train_total_loss is 0.15629518 val_total_loss is 0.3322241
learning_rate is  0.001
The 13700 step train_total_loss is 0.5431278 val_total_loss is 1.2893933
learning_rate is  0.001
The 13750 step train_total_loss is 0.18644491 val_total_loss is 0.06427208
learning_rate is  0.001
The 13800 step train_total_loss is 0.5305901 val_total_loss is 0.62942076
learning_rate is  0.001
The 13850 step train_total_loss is 0.9389546 val_total_loss is 0.4423432
learning_rate is  0.001
The 13900 step train_total_loss is 0.093795136 val_total_loss is 0.15144095
learning_rate is  0.001
The 13950 step train_total_loss is 0.5573693 val_total_loss is 0.2006041
learning_rate is  0.001
The 14000 step train_total_loss is 0.785847 val_total_loss is 0.16619295
learning_rate is  0.001
The 14050 step train_total_loss is 0.1358985 val_total_loss is 0.12109211
learning_rate is  0.001
The 14100 step trai

The 17800 step train_total_loss is 0.35269853 val_total_loss is 1.0010704
learning_rate is  0.001
The 17850 step train_total_loss is 0.27998966 val_total_loss is 0.7720747
learning_rate is  0.001
The 17900 step train_total_loss is 0.8983877 val_total_loss is 0.22174071
learning_rate is  0.001
The 17950 step train_total_loss is 0.3560379 val_total_loss is 0.16261162
learning_rate is  0.001
The 18000 step train_total_loss is 0.027501777 val_total_loss is 0.4693451
learning_rate is  0.001
The 18050 step train_total_loss is 0.41542006 val_total_loss is 0.16683175
learning_rate is  0.001
The 18100 step train_total_loss is 0.38567743 val_total_loss is 0.13804625
learning_rate is  0.001
The 18150 step train_total_loss is 0.33153722 val_total_loss is 0.6393812
learning_rate is  0.001
The 18200 step train_total_loss is 0.8749365 val_total_loss is 0.27410108
learning_rate is  0.001
The 18250 step train_total_loss is 0.91022134 val_total_loss is 0.09544652
learning_rate is  0.001
The 18300 step t

The 22000 step train_total_loss is 0.12794423 val_total_loss is 0.048470076
learning_rate is  0.001
The 22050 step train_total_loss is 0.10955246 val_total_loss is 0.13686475
learning_rate is  0.001
The 22100 step train_total_loss is 0.2300353 val_total_loss is 0.072089754
learning_rate is  0.001
The 22150 step train_total_loss is 0.045761697 val_total_loss is 0.5292311
learning_rate is  0.001
The 22200 step train_total_loss is 0.13739507 val_total_loss is 0.18984838
learning_rate is  0.001
The 22250 step train_total_loss is 0.30189884 val_total_loss is 0.49797082
learning_rate is  0.001
The 22300 step train_total_loss is 0.32237354 val_total_loss is 0.41024148
learning_rate is  0.001
The 22350 step train_total_loss is 0.37028933 val_total_loss is 0.37945473
learning_rate is  0.001
The 22400 step train_total_loss is 0.90408015 val_total_loss is 0.16049193
learning_rate is  0.001
The 22450 step train_total_loss is 0.20764472 val_total_loss is 0.15882277
learning_rate is  0.001
The 22500